In [1]:
heartsa = [str(i) + 'H' for i in range(7,15)]
clubsa = [str(i) + 'C' for i in range(7,15)]
spadesa = [str(i) + 'S' for i in range(7,15)]
diamondsa = [str(i) + 'D' for i in range(7,15)]

allcardsa = heartsa + clubsa + spadesa + diamondsa
valuesa = [0, 0, 0, 10, 2, 3, 4, 11, 0, 0, 0, 10, 2, 3, 4, 11, 0, 0, 0, 10, 2, 3, 4, 11, 0, 0, 0, 10, 2, 3, 4, 11]

pointvalues = dict(zip(allcardsa, valuesa))

# card types
hearts = [str(i) + 'H' for i in range(7, 11)] + ['13H'] + ['14H']
spades = [str(i) + 'S' for i in range(7, 11)] + ['13S'] + ['14S']
clubs = [str(i) + 'C' for i in range(7, 11)] + ['13C'] + ['14C']
# Trump
trump = [str(i) + 'D' for i in range(7, 15)] + ['11' + i for i in ['H', 'S', 'C']] + ['12' + i for i in ['H', 'S', 'C']]

fail = hearts + spades + clubs

points = [0, 0, 0, 10, 4, 11]
pointstrump = [0, 0, 0, 10, 2, 3, 4, 11, 2, 2, 2, 3, 3, 3]
trumprank = [1, 2, 3, 5, 7, 11, 4, 6, 8, 9, 10, 12, 13, 14]
failrank = [1, 2, 3, 5, 4, 6]

faildic = dict(zip(fail, zip(points * 4, failrank * 4)))
trumpdic = dict(zip(trump, zip(pointstrump, trumprank)))

In [2]:
import pandas as pd
import numpy as np
# ipython autotime
%load_ext autotime

# ipython_memory_usage
import ipython_memory_usage 
%ipython_memory_usage_start  
from pprint import pprint as pp

time: 124 ms
In [2] used 0.0078 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 101.52 MiB


In [3]:
dfnew = pd.read_pickle('2400000_2400800')

time: 411 ms
In [3] used 19.0000 MiB RAM in 0.52s, peaked 0.00 MiB above current, total RAM usage 120.52 MiB


In [4]:
df = dfnew.copy()

time: 4.47 ms
In [4] used -2.3594 MiB RAM in 0.10s, peaked 2.37 MiB above current, total RAM usage 118.16 MiB


In [5]:
df['cards'] = df[df.columns[1:7]].apply(lambda x: ','.join(x.astype(str)).split(','), axis=1)
df['trump'] = df['cards'].apply(lambda x: [i for i in x if i in trumpdic.keys()])
df['trumpcount'] = df['trump'].apply(lambda x: len(x))
df['fail'] = df['cards'].apply(lambda x: [i for i in x if i in faildic.keys()])
df['failcount'] = df['fail'].apply(lambda x: len(x))
df['heldpoints'] = df['cards'].apply(lambda x: sum([pointvalues[i] for i in x]))
df['outcome'] = df['score'].apply(lambda x: 'win' if x > 0 else 'lose')
df['uniqueid'] = df[['handid', 'name']].astype(str).apply('-'.join,1)


time: 516 ms
In [5] used 4.7422 MiB RAM in 0.62s, peaked 0.00 MiB above current, total RAM usage 122.90 MiB


In [6]:
dfa = df[['uniqueid', 1, 2, 3, 4, 5, 6, 'score', 'picker', 'pick_position', 'trumpcount', 'heldpoints', 'outcome']]

time: 7.23 ms
In [6] used 0.5273 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 123.43 MiB


In [7]:
pickerhands = dfa['picker'] == 'picker'
dfa[pickerhands]

,uniqueid,1,2,3,4,5,6,score,picker,pick_position,trumpcount,heldpoints,outcome
3,2400001-RecordHolder37,7S,11C,12S,12C,11H,11D,16,picker,2,5,12,win
2,2400002-TSG Jerry,7H,11S,12D,11H,12H,12S,8,picker,2,5,13,win
4,2400003-Dave Frederick,13D,12C,8D,11H,9S,12D,2,picker,3,5,12,win
4,2400004-Shawn Theune,14D,7D,10D,8H,12S,11S,2,picker,3,5,26,win
1,2400005-TSG Dan,7H,11D,9H,7D,12S,12C,8,picker,1,4,8,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2400795-vtreague,9D,11S,9C,13C,11C,12H,2,picker,0,4,11,win
2,2400796-jerryklein1341,14D,9D,9S,10D,12H,11C,2,picker,3,5,26,win
1,2400797-eugcindy,12C,12S,14C,13H,11S,9H,2,picker,0,3,23,win
2,2400798-drdistilledjd,13D,9D,11C,11D,12C,9S,-8,picker,4,5,11,lose


time: 93.9 ms
In [7] used 0.1328 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 123.56 MiB


In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

from pipeline_modules import average_metrics_pipeline
from pipeline_modules import classification_reports_pipeline
from pipeline_modules import roc_curve_pipeline

time: 1.88 s
In [8] used 67.8047 MiB RAM in 1.99s, peaked 0.00 MiB above current, total RAM usage 191.37 MiB


In [9]:
df = dfa.copy()


time: 5.31 ms
In [9] used 0.2695 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 191.64 MiB


In [10]:
df['outcome'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)

time: 5.03 ms
In [10] used 0.2227 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 191.86 MiB


In [11]:
pickerdf = (df['picker'] == 'picker')
dfm = df[pickerdf]
dfm.drop(['picker'], axis=1, inplace=True)

time: 4.74 ms
In [11] used 0.0625 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 191.92 MiB


In [13]:
dfm.reset_index(inplace=True)

time: 1.9 ms
In [13] used 0.0039 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 192.03 MiB


In [14]:
dfmx = dfm[['uniqueid',               1,               2,               3,
                     4,               5,               6,         'score',
       'pick_position',    'trumpcount',    'heldpoints']]

       

time: 2.75 ms
In [14] used 0.0234 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 192.05 MiB


In [15]:
dfmx

,uniqueid,1,2,3,4,5,6,score,pick_position,trumpcount,heldpoints
0,2400001-RecordHolder37,7S,11C,12S,12C,11H,11D,16,2,5,12
1,2400002-TSG Jerry,7H,11S,12D,11H,12H,12S,8,2,5,13
2,2400003-Dave Frederick,13D,12C,8D,11H,9S,12D,2,3,5,12
3,2400004-Shawn Theune,14D,7D,10D,8H,12S,11S,2,3,5,26
4,2400005-TSG Dan,7H,11D,9H,7D,12S,12C,8,1,4,8
...,...,...,...,...,...,...,...,...,...,...,...
693,2400795-vtreague,9D,11S,9C,13C,11C,12H,2,0,4,11
694,2400796-jerryklein1341,14D,9D,9S,10D,12H,11C,2,3,5,26
695,2400797-eugcindy,12C,12S,14C,13H,11S,9H,2,0,3,23
696,2400798-drdistilledjd,13D,9D,11C,11D,12C,9S,-8,4,5,11


time: 17.8 ms
In [15] used 0.0117 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 192.07 MiB


In [16]:
dfmy = dfm['outcome']

time: 1.4 ms
In [16] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 192.07 MiB


In [17]:
dfmx = pd.get_dummies(dfmx, columns=[1, 2, 3, 4, 5, 6,])

time: 12.5 ms
In [17] used 0.3008 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 192.37 MiB


In [18]:
dfma = pd.concat([dfmx, dfmy], axis=1)

time: 4.33 ms
In [18] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 192.37 MiB


In [19]:
X = dfma.iloc[:,2:-1]
y = dfma.iloc[:,-1]

time: 2.6 ms
In [19] used 0.0078 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 192.38 MiB


In [20]:
dfma.to_pickle('new_hands_df')

time: 3.21 ms
In [20] used 0.1797 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 192.55 MiB


In [21]:
dfma

,uniqueid,score,pick_position,trumpcount,heldpoints,1_10C,1_10D,1_10H,1_10S,1_11C,...,6_7S,6_8C,6_8D,6_8H,6_8S,6_9C,6_9D,6_9H,6_9S,outcome
0,2400001-RecordHolder37,16,2,5,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2400002-TSG Jerry,8,2,5,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2400003-Dave Frederick,2,3,5,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2400004-Shawn Theune,2,3,5,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2400005-TSG Dan,8,1,4,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,2400795-vtreague,2,0,4,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
694,2400796-jerryklein1341,2,3,5,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
695,2400797-eugcindy,2,0,3,23,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
696,2400798-drdistilledjd,-8,4,5,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


time: 22 ms
In [21] used 0.0117 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 192.57 MiB
